In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import random
import skfuzzy as fuzz
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
data = pd.read_excel('train_norm.xlsx')

In [ ]:
data.set_index('Unnamed: 0',inplace=True)
data.head()

In [ ]:
X=np.array(data.iloc[:,0:45])
y=np.array(data.iloc[:,45])
X.shape

In [ ]:
rules = [] 
while(len(set(rules))!= 1000):
    res=''.join(random.choices(['1','2','3','#'], weights = [7.5, 10, 7.5, 5], k = 18))
    rules.append(res)

In [ ]:
rules = [] 

def train_string(temp):
    X=[]
    for i in range(temp.shape[0]):
        r=''
        for j in range(temp.shape[1]):
            a1=mem(temp[i,j],'1')
            a2=mem(temp[i,j],'2')
            a3=mem(temp[i,j],'3')
            print(a1,a2,a3)
            if (a1>=a2) and (a1>=a3):
                r=r+'1'
            elif (a2>=a1) and (a2>=a3):
                r=r+'2'
            elif (a3>=a1) and (a3>=a2):
                r=r+'3'
        X.append(r)
    return X
    

In [ ]:
rules=train_string(X)

In [ ]:
rules=train_string(X)
rules=set(rules)
t=set(rules)
while True:
    for rule in rules:
        temp=rule
        for j in range(len(rule)):
            a=random.random()
            if a<.2:
                temp=temp[0:j]+'#'+temp[j+1:]
        t.add(temp)
    if len(t)>=500:
            break
rules=list(t)

In [ ]:
a1=fuzz.trimf(np.arange(0,1.1,0.1),[0,0,0.5])
a2=fuzz.trimf(np.arange(0,1.1,0.1),[0,0.5,1.0])
a3=fuzz.trimf(np.arange(0,1.1,0.1),[0.5,0.5,1.0])
def mem(x,rule):
    if rule=='1':
        return fuzz.interp_membership(np.arange(0,1.1,0.1), a1, x)
    elif rule=='2':
        return fuzz.interp_membership(np.arange(0,1.1,0.1), a2, x)
    elif rule=='3':
        return fuzz.interp_membership(np.arange(0,1.1,0.1), a3, x)
    else:
        return 1

In [ ]:
def com_grade(l,rule):
    res=1
    for i in range(len(l)):
        res=res*mem(l[i],rule[i])
        if res==0:
            break
    return res

In [ ]:
def grade_certainty(X,rules,y):
    certainty_factor=[]
    cons=[]
    fitness=[]
    for i in range(len(rules)):
        t=[]
        for j in range(X.shape[0]):
            temp=com_grade(X[j],rules[i])
            t.append(temp)
        Bclass0=0
        Bclass1=0
        for j in range(len(t)):
            if y[j]==0:
                Bclass0+=t[j]
            else:
                Bclass1+=t[j]
        cons_class=max(Bclass0,Bclass1)
        o_class=min(Bclass0,Bclass1)
        if cons_class==0 and o_class==0:
            cf=0
        else:
            cf=(cons_class - o_class)/(cons_class + o_class)
        if cons_class==Bclass1:
            cons.append(1)
            ncp=0
            for j in range(len(t)):
                if (y[j]==1) and (t[j]!=0):
                    ncp+=1 
            fitness.append(ncp)
        else:
            cons.append(0)
            ncp=0
            for j in range(len(t)):
                if (y[j]==0) and (t[j]!=0):
                    ncp+=1 
            fitness.append(ncp)
        certainty_factor.append(cf)
    return cons,certainty_factor,fitness

In [ ]:
def selectOne(population,fitness):
    max = sum([fitness[c] for c in range(len(population))])
    selection_probs = [fitness[c]/max for c in range(len(population))]
    return population[np.random.choice(len(population), p=selection_probs)]

In [ ]:
def cxTwoPoint(ind1, ind2):
    ind1=[i for i in ind1]
    ind2=[i for i in ind2]
    size = min(len(ind1), len(ind2))
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:  # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] = ind2[cxpoint1:cxpoint2], ind1[cxpoint1:cxpoint2]
    ind1=''.join([str(elem) for elem in ind1])
    ind2=''.join([str(elem) for elem in ind2])
    return ind1, ind2

In [ ]:
def predict(X,y,rules,cons_class,certainty_factor):
    output_class=[]
    cf=[]
    r=[]
    for i in range(X.shape[0]):
        m=-1
        o=None
        for j in range(len(rules)):
            com=com_grade(X[i],rules[j])
            if com*certainty_factor[j]>m:
                m=com*certainty_factor[j]
                o=cons_class[j]
                ru=rules[j]
        cf.append(m)
        output_class.append(o)
        r.append(ru)
    return output_class,cf,r

In [ ]:
def model(X,y,rules,epoch,X_test,y_test):
    while epoch:
        cons,cf,fitness=grade_certainty(X,rules,y)
        cons_class=[]
        certainty_factor=[]
        rev_rules=[]
        fit=[]
        for i in range(len(cf)):
            if cf[i]!=0:
                certainty_factor.append(cf[i])
                cons_class.append(cons[i])
                rev_rules.append(rules[i])
                fit.append(fitness[i])
        output_class,cf,r=predict(X_test,y_test,rev_rules,cons_class,certainty_factor)
        print(epoch, accuracy_score(y_test,output_class))
        zipped_pairs = zip(fit, cons_class, certainty_factor, rev_rules) 
        temp=sorted(zipped_pairs,reverse=True)
        i=0
        for a,b,c,d in temp:
            fit[i]=a
            cons_class[i]=b
            certainty_factor[i]=c 
            rev_rules[i]=d 
            i+=1 
        rules_rep=[]
        for i in range(len(rev_rules)//4):
            ind1=selectOne(rev_rules,fit)
            ind2=selectOne(rev_rules,fit)
            ind1,ind2=cxTwoPoint(ind1, ind2)
            rules_rep.append(ind1)
            rules_rep.append(ind2)
        rules=rev_rules[:len(rev_rules)//2]+rules_rep
        epoch-=1
    return cons_class,certainty_factor,rev_rules,fit